# Preparation

Import required dependencies to use Google's API

In [4]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

Define the accesses to be granted

In [5]:
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', # View your Google Classroom classes.
          'https://www.googleapis.com/auth/classroom.rosters', # Manage your Google Classroom class rosters.
          'https://www.googleapis.com/auth/classroom.profile.emails', # View the email addresses of people in your classes.
          'https://www.googleapis.com/auth/classroom.topics', #See, create, and edit topics in Google Classroom.
          'https://www.googleapis.com/auth/classroom.coursework.students', # Manage coursework and grades for students
          'https://www.googleapis.com/auth/classroom.courseworkmaterials', # See, edit, and create classwork materials in Google Classroom.
          'https://www.googleapis.com/auth/spreadsheets.readonly'] # See all your Google Sheets spreadsheets.

Use the provided `token.json` file to authenticate and access user data, if `token.json` is not available, it would be created from the provided `credentials.json` by logging into user's Google Account

In [6]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=845305684945-mm42jq9nkk30ov1qf64ur3t855jp0e27.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55306%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courses.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.rosters+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.profile.emails+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.topics+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.coursework.students+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courseworkmaterials+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=h5AIM4xpbhWim5JR9kYOGEzIFwQB2A&access_type=offline


Call the Google Classroom API to access various methods with user's access from the credential that has been authenticated

In [7]:
service = build('classroom', 'v1', credentials=creds)

# Course
In this section, the user is going to choose which Google Classroom Course that is going to be accessed

Use the `courses().list()` method to show a list of the user's courses

In [8]:
results = service.courses().list(pageSize=5).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 5 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- Testing DV
- Kementerian BUMN
- Automation
- Wizard Data Analytics
- Vulcan Machine Learning


Input the course name (**case-insensitive**) that is going to be accessed (Example: `wizard data visualization`). If the course was found, the course's `id` will be retrieved.

In [9]:
course_input = input("Enter Course Name: ")
course_lowercase = course_input.lower()
course_id = None

for course in courses:
    if course_lowercase == course['name'].lower():
        course_id = course['id']
        break

if course_id == None:
    raise Exception(f"{course_input} course not found")

else:
    print(f'{course_input} found with ID {course_id}')

testing dv found with ID 621398536690


# Topic
In this section, the user is going to create new topics to group the courseworks and materials

Define the topics names in a list

In [8]:
topic_list = ['Capstone Project', 'Learning by Building', 
              'Quiz', 'Additional Assignment', 'Lecture Notes',
              'Material', 'Administration Deck']

Create the topics using `topics().create()` method, if the topic was already created, it would show `Requested entity already exists` error.

In [9]:
try:
  for name in topic_list:
    topic = {
        "name": name
    }
    response = service.courses().topics().create(
      courseId=course_id,
      body=topic).execute()

    print('Topic created: {} - {}'.format(response['name'], response['topicId']))

except HttpError as error:
  print(f"An error occurred: {error}")

Topic created: Capstone Project - 621385140075
Topic created: Learning by Building - 621387154997
Topic created: Quiz - 621385910017
Topic created: Additional Assignment - 621385190981
Topic created: Lecture Notes - 621385150508
Topic created: Material - 621385910071
Topic created: Administration Deck - 621385692410


Use the `topics().list()` method to show a list of the topics that have been created

In [13]:
topics = []

response = service.courses().topics().list(
    courseId=course_id).execute()
topics.extend(response.get('topic', []))

print('Topics : ')
for topic in topics:
    print(' - {} ({})'.format(topic['name'], topic['topicId']))

Topics : 
 - Administration Deck (621385692410)
 - Material (621385910071)
 - Lecture Notes (621385150508)
 - Additional Assignment (621385190981)
 - Quiz (621385910017)
 - Learning by Building (621387154997)
 - Capstone Project (621385140075)


# Administration Deck

In the section, the user is going to post courseworks and materials that belongs to the topic `Administration Deck`

To access the descriptions of each courseworks and materials, the user can use the `Classwork()` object from `assets.py` that store the material's descriptions as attribute.

In [11]:
from assets import Classwork

description = Classwork()

Retrieve the corresponding topic's id

In [14]:
for topic in topics:
    if topic['name'] == 'Administration Deck':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '621398536690',
 'topicId': '621385692410',
 'name': 'Administration Deck',
 'updateTime': '2023-09-06T10:45:44.175Z'}

Create the `Permohonan Sesi Mentoring` post

In [13]:
coursework = {
    'title': 'Permohonan Sesi Mentoring',
    'description': description.administration_deck['Permohonan Sesi Mentoring'],
    'materials': [
        {'link': {'url': 'http://bit.ly/mentoring-onsite?authuser=0'}},
        {'link': {'url': 'https://github.com/teamalgoritma/community?authuser=0'}},
        {'link': {'url': 'https://drive.google.com/file/d/1SuU3e8hg9jfMO25kWang1MIaaePa2x-Y/view?usp=drive_web&authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material Permohonan Sesi Mentoring created


Create the `How to Ask Questions` post

In [14]:
coursework = {
    'title': 'How to Ask Questions',
    'description': description.administration_deck['How to Ask Questions'],
    'materials': [
        {'link': {'url': 'https://github.com/teamalgoritma/community?authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material How to Ask Questions created


Create the `Administration Deck` post

In [15]:
coursework = {
    'title': 'Administration Deck',
    'description': description.administration_deck['Administration Deck'],
    'materials': [
        {'link': {'url': 'https://drive.google.com/file/d/1ivjryti41IekS55HSj43bC8TXLaZZrii/view?usp=drive_web&authuser=0'}},
        {'link': {'url': 'https://drive.google.com/file/d/1vUG9v_Aie-aHW4U93FD7LKcjfxilHG5u/view?usp=drive_web&authuser=0'}}
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material Administration Deck created


# Material
In the section, the user is going to post courseworks and materials that belongs to the topic `Material`

Retrieve the corresponding topic's id

In [16]:
for topic in topics:
    if topic['name'] == 'Material':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '621398536690',
 'topicId': '621385910071',
 'name': 'Material',
 'updateTime': '2023-09-06T10:45:33.006Z'}

Create the `4. M : Interactive Plotting (IP)` post

In [17]:
coursework = {
    'title': '4. M : Interactive Plotting (IP)',
    'materials': [
        {'link': {'url': 'https://bit.ly/ip_material?authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 4. M : Interactive Plotting (IP) created


Create the `3. M : Data Visualization (DV)` post

In [18]:
coursework = {
    'title': '3. M : Data Visualization (DV)',
    'materials': [
        {'link': {'url': 'https://bit.ly/dviz_material?authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 3. M : Data Visualization (DV) created


Create the `2. M : Practical Statistics (PS)` post

In [19]:
coursework = {
    'title': '2. M : Practical Statistics (PS)',
    'materials': [
        {'link': {'url': 'https://bit.ly/ps_material?authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 2. M : Practical Statistics (PS) created


Create the `1. M : Programming for Data Science (P4DS)` post

In [20]:
coursework = {
    'title': '1. M : Programming for Data Science (P4DS)',
    'materials': [
        {'link': {'url': 'https://bit.ly/p4ds_material?authuser=0'}},
    ],
    'topicId' : topic_id,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 1. M : Programming for Data Science (P4DS) created


# Quiz
In the section, the user is going to post courseworks and materials that belongs to the topic `Quiz`

Retrieve the corresponding topic's id

In [15]:
for topic in topics:
    if topic['name'] == 'Quiz':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '621398536690',
 'topicId': '621385910017',
 'name': 'Quiz',
 'updateTime': '2023-09-06T10:46:00.563Z'}

Create the `3. Q: Interactive Plotting (IP)` post

In [22]:
coursework = {
    'title': '3. Q: Interactive Plotting (IP)',
    'description': description.quiz['3. Q: Interactive Plotting (IP)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/quizzes/interactive-plotting-quiz/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 1,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 3. Q: Interactive Plotting (IP) created


Create the `2. Q: Data Visualization (DV)` post

In [23]:
coursework = {
    'title': '2. Q: Data Visualization (DV)',
    'description': description.quiz['2. Q: Data Visualization (DV)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/courses/data-visualization-in-r/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 2,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 2. Q: Data Visualization (DV) created


Create the `1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS)` post

In [16]:
coursework = {
    'title': '1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS)',
    'description': description.quiz['1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/courses/practical-statistics-for-data-science/?authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 4,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS) created


Create the `0. Q: Prerequisite Check` post

In [25]:
coursework = {
    'title': '0. Q: Prerequisite Check',
    'description': description.quiz['0. Q: Prerequisite Check'],
    'materials': [
        {'link': {'url': 'https://drive.google.com/file/d/1fkU61MnkI3H56AFpqrvnMPBUtMmepTT5/view?usp=drive_web&authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 0. Q: Prerequisite Check created


# Learning by Building
In the section, the user is going to post courseworks and materials that belongs to the topic `Learning by Building`

Retrieve the corresponding topic's id

In [26]:
for topic in topics:
    if topic['name'] == 'Learning by Building':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '621398536690',
 'topicId': '621387154997',
 'name': 'Learning by Building',
 'updateTime': '2023-09-06T10:45:26.610Z'}

Create the `3. LBB: Interactive Plotting` post

In [27]:
coursework = {
    'title': '3. LBB: Interactive Plotting',
    'description': description.lbb['3. LBB: Interactive Plotting'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 3,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 3. LBB: Interactive Plotting created


Create the `2. LBB: Data Visualization` post

In [28]:
coursework = {
    'title': '2. LBB: Data Visualization',
    'description': description.lbb['2. LBB: Data Visualization'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 2. LBB: Data Visualization created


Create the `1. LBB: Programming for Data Science` post

In [29]:
coursework = {
    'title': '1. LBB: Programming for Data Science',
    'description': description.lbb['1. LBB: Programming for Data Science'],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment 1. LBB: Programming for Data Science created


# Capstone Project
In the section, the user is going to post courseworks and materials that belongs to the topic `Capstone Project`

Retrieve the corresponding topic's id

In [30]:
for topic in topics:
    if topic['name'] == 'Capstone Project':
        topic_id = topic['topicId']
        break 

topic

{'courseId': '621398536690',
 'topicId': '621385140075',
 'name': 'Capstone Project',
 'updateTime': '2023-09-06T10:45:26.079Z'}

Create the `Brief Capstone Data Visualization` post

In [31]:
coursework = {
    'title': 'Brief Capstone Data Visualization',
    'description': description.capstone['Brief Capstone Data Visualization'],
    'materials': [
        {'link': {'url': 'https://rpubs.com/Algoritma/dv-capstone'}},
        {'link': {'url': 'https://drive.google.com/file/d/1KQU0f2lH9HaMA1EK1DqBgEehTzyZDvxr/view?usp=drive_web&authuser=0'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment Brief Capstone Data Visualization created


Create the `Capstone Data Visualization` post

In [32]:
coursework = {
    'title': 'Capstone Data Visualization',
    'description': description.capstone['Capstone Data Visualization'],
    'materials': [
        {'link': {'url': 'https://rpubs.com/Algoritma/dv-capstone'}}
    ],
    'topicId' : topic_id,
    'workType': 'ASSIGNMENT',
    'maxPoints': 30,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

Assignment Capstone Data Visualization created
